<a href="https://colab.research.google.com/github/SONRAJAPU/CV/blob/main/DL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sure, here are the steps to implement this project using the CIFAR-10 dataset, ResNet architecture, and FGSM attack:

Load the CIFAR-10 dataset and preprocess the data (e.g., normalize the images).
Define the ResNet model architecture and train it on the CIFAR-10 dataset. You can use a pre-trained ResNet model if you prefer.

Implement the FGSM attack using a library like PyTorch or TensorFlow. FGSM requires calculating the gradient of the loss function with respect to the input and using it to generate adversarial examples by adding a small perturbation to the input.

Generate adversarial examples for a subset of the CIFAR-10 dataset using the FGSM attack. You can use different strengths of the attack by adjusting the magnitude of the perturbation.

Evaluate the accuracy and loss of the ResNet model on the clean CIFAR-10 dataset and the adversarial examples generated by the FGSM attack.

Dissect the ResNet model into multiple components, such as the convolutional layers, the fully connected layers, and the skip connections.

Freeze the parameters in each component except for the ones you want to fine-tune for adversarial robustness. For example, you can freeze the parameters in the convolutional layers and the skip connections and fine-tune the fully connected layers.

Fine-tune the selected components of the ResNet model using the adversarial examples generated in Step 4. You can use a small learning rate and a few epochs to avoid overfitting.
Evaluate the accuracy and loss of the fine-tuned ResNet model on the clean CIFAR-10 dataset and the adversarial examples generated by the FGSM attack.
Compare the loss values of each component to understand their contribution to the overall performance drop during adversarial attacks. You can use visualization techniques like heatmaps or saliency maps to visualize the importance of each component in the network's predictions.
Analyze the results and draw conclusions about the sensitivity of the ResNet model to adversarial attacks and the contribution of each component to its robustness.

Load the CIFAR-10 dataset and preprocess the data (e.g., normalize the images). Define the ResNet model architecture and train it on the CIFAR-10 dataset. You can use a pre-trained ResNet model if you prefer. 



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Define transforms for the data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=transform_train)
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform_test)

# Define the dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

# Load the pre-trained ResNet model and modify the output layer to fit the CIFAR-10 dataset
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


100%|██████████| 170498071/170498071 [00:12<00:00, 13411516.99it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 282MB/s]


[1,   100] loss: 1.440
[1,   200] loss: 1.011
[1,   300] loss: 0.899
[2,   100] loss: 0.754
[2,   200] loss: 0.707
[2,   300] loss: 0.699
[3,   100] loss: 0.623
[3,   200] loss: 0.610
[3,   300] loss: 0.618
[4,   100] loss: 0.553
[4,   200] loss: 0.567
[4,   300] loss: 0.558
[5,   100] loss: 0.518
[5,   200] loss: 0.504
[5,   300] loss: 0.524
[6,   100] loss: 0.486
[6,   200] loss: 0.489
[6,   300] loss: 0.490
[7,   100] loss: 0.448
[7,   200] loss: 0.442
[7,   300] loss: 0.457
[8,   100] loss: 0.409
[8,   200] loss: 0.433
[8,   300] loss: 0.437
[9,   100] loss: 0.393
[9,   200] loss: 0.402
[9,   300] loss: 0.413
[10,   100] loss: 0.396
[10,   200] loss: 0.375
[10,   300] loss: 0.394
Finished Training


Implement the FGSM attack using a library like PyTorch or TensorFlow. FGSM requires calculating the gradient of the loss function with respect to the input and using it to generate adversarial examples by adding a small perturbation to the input.


In [3]:
import torch

def fgsm_attack(model, loss, x, y, epsilon):
    # Set requires_grad attribute of tensor x
    x.requires_grad = True
    
    # Forward pass
    output = model(x)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    
    # If the initial prediction is wrong, don't bother attacking
    if init_pred.item() != y.item():
        return x
    
    # Calculate the loss
    loss = loss(output, y)
    
    # Zero all existing gradients
    model.zero_grad()
    
    # Calculate gradients of model in backward pass
    loss.backward()
    
    # Collect datagrad
    data_grad = x.grad.data
    
    # Call sign() on the gradients to get the sign of the gradient
    sign_data_grad = data_grad.sign()
    
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = x + epsilon*sign_data_grad
    
    # Clip the perturbed image to keep it within the range [0,1]
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    
    # Return the perturbed image
    return perturbed_image


Generate adversarial examples for a subset of the CIFAR-10 dataset using the FGSM attack. You can use different strengths of the attack by adjusting the magnitude of the perturbation.

In [5]:
pip3 install torch==1.3.1+cpu torchvision==0.4.2+cpu -f 

SyntaxError: ignored

In [6]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

def fgsm_attack(model, loss, x, y, epsilon):
    # Generate perturbation
    delta = torch.zeros_like(x, requires_grad=True)
    loss_val = loss(model(x + delta), y)
    loss_val.backward()
    delta.data = epsilon * delta.grad.detach().sign()
    
    # Add perturbation to input
    x_adv = x + delta
    
    # Clip perturbed input to valid range
    x_adv = torch.clamp(x_adv, 0, 1)
    
    return x_adv

# Define transforms for the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the CIFAR-10 dataset
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)

# Define the dataloader
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Define the pre-trained ResNet model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# Set the model to eval mode
model.eval()

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Generate adversarial examples for the first 100 images in the test set
epsilon = 0.1
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs.requires_grad = True
    
    # Generate adversarial examples using FGSM attack
    adv_inputs = fgsm_attack(model, criterion, inputs, labels, epsilon)
    
    # Evaluate accuracy on original and adversarial examples
    outputs = model(inputs)
    _, pred = torch.max(outputs.data, 1)
    correct = (pred == labels).sum().item()
    acc = correct / labels.size(0)
    
    adv_outputs = model(adv_inputs)
    _, adv_pred = torch.max(adv_outputs.data, 1)
    adv_correct = (adv_pred == labels).sum().item()
    adv_acc = adv_correct / labels.size(0)
    
    print('Batch %d: Original Acc = %.2f%%, Adversarial Acc = %.2f%%' % (i+1, acc*100, adv_acc*100))
    
    if i == 0:
        # Save the original and adversarial examples for visualization
        torchvision.utils.save_image(inputs, 'original.png')
        torchvision.utils.save_image(adv_inputs, 'adversarial.png')


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Batch 1: Original Acc = 7.00%, Adversarial Acc = 8.00%
Batch 2: Original Acc = 4.00%, Adversarial Acc = 3.00%
Batch 3: Original Acc = 6.00%, Adversarial Acc = 6.00%
Batch 4: Original Acc = 5.00%, Adversarial Acc = 2.00%
Batch 5: Original Acc = 4.00%, Adversarial Acc = 3.00%
Batch 6: Original Acc = 5.00%, Adversarial Acc = 5.00%
Batch 7: Original Acc = 10.00%, Adversarial Acc = 7.00%
Batch 8: Original Acc = 6.00%, Adversarial Acc = 5.00%
Batch 9: Original Acc = 8.00%, Adversarial Acc = 2.00%
Batch 10: Original Acc = 9.00%, Adversarial Acc = 8.00%
Batch 11: Original Acc = 7.00%, Adversarial Acc = 8.00%
Batch 12: Original Acc = 4.00%, Adversarial Acc = 7.00%
Batch 13: Original Acc = 9.00%, Adversarial Acc = 6.00%
Batch 14: Original Acc = 8.00%, Adversarial Acc = 5.00%
Batch 15: Original Acc = 5.00%, Adversarial Acc = 4.00%
Batch 16: Original Acc = 8.00%, Adversarial Acc = 7.00%
Batch 17: Original Acc = 6.00%, Adversarial Acc = 6.00%
Batch 18: Original Acc = 12.00%, Adversarial Acc = 6.00%

In [11]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

def fgsm_attack(model, loss, x, y, epsilon):
    x = torch.tensor(x, requires_grad=True)
    # Generate perturbation
    delta = torch.zeros_like(x, requires_grad=True)
    torch.set_grad_enabled(True)  # Context-manager 
    loss_val = loss(model(x + delta), y)
    loss_val.backward()
    delta.data = epsilon * delta.grad.detach().sign()
    
    # Add perturbation to input
    x_adv = x + delta
    
    # Clip perturbed input to valid range
    x_adv = torch.clamp(x_adv, 0, 1)
    
    return x_adv

# Define transforms for the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the CIFAR-10 dataset
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)

# Define the dataloader
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

# Define the pre-trained ResNet model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to eval mode
model.eval()

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

# Generate adversarial examples for the test set
epsilon = 0.1
total = 0
correct = 0
adv_correct = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Evaluate accuracy on original examples
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        correct += (pred == labels).sum().item()
        
        # Generate adversarial examples using FGSM attack
        adv_inputs = fgsm_attack(model, criterion, inputs, labels, epsilon)
        adv_inputs = adv_inputs.to(device)
        
        # Evaluate accuracy on adversarial examples
        adv_outputs = model(adv_inputs)
        _, adv_pred = torch.max(adv_outputs.data, 1)
        adv_correct += (adv_pred == labels).sum().item()
        
        total += labels.size(0)
    
    # Print accuracy on original and adversarial examples
    acc = 100 * correct / total
    adv_acc = 100 * adv_correct / total
    print('Original Acc = %.2f%%, Adversarial Acc = %.2f%%' % (acc, adv_acc))
    
    # Save the original and adversarial examples for visualization
   


Files already downloaded and verified


<ipython-input-11-a4ee0be1f79d>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, requires_grad=True)


Original Acc = 8.39%, Adversarial Acc = 7.88%


In [10]:
# Generate adversarial examples for the test set
epsilon = 0.1
total = 0
correct = 0
adv_correct = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Evaluate accuracy on original examples
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        correct += (pred == labels).sum().item()
        
        # Set the requires_grad flag to True for the input tensor
        inputs.requires_grad = True
        
        # Generate adversarial examples using FGSM attack
        adv_inputs = fgsm_attack(model, criterion, inputs, labels, epsilon)
        adv_inputs = adv_inputs.to(device)
        
        # Evaluate accuracy on adversarial examples
        adv_outputs = model(adv_inputs)
        _, adv_pred = torch.max(adv_outputs.data, 1)
        adv_correct += (adv_pred == labels).sum().item()
        
        total += labels.size(0)
    
    # Print accuracy on original and adversarial examples
    acc = 100 * correct / total
    adv_acc = 100 * adv_correct / total
    print('Original Acc = %.2f%%, Adversarial Acc = %.2f%%' % (acc, adv_acc))
    
    # Save the original and adversarial examples for visualization


<ipython-input-9-9d3b9083f90d>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, requires_grad=True)


RuntimeError: ignored